In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [4]:
# --- 1. Fonction de Scrapping (S&P 500 uniquement) ---

def get_sp500_tickers():
    """
    Récupère la liste des tickers du S&P 500 depuis Wikipedia.
    """
    print("Récupération des tickers S&P 500...")
    tickers = []
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'id': 'constituents'})

        if table:
            for row in table.find_all('tr')[1:]: # Ignorer l'en-tête
                cells = row.find_all('td')
                if cells:
                    ticker = cells[0].text.strip()
                    # Remplacement pour yfinance (ex: BRK.B -> BRK-B)
                    ticker = ticker.replace('.', '-')
                    tickers.append(ticker)

        print(f"Trouvé {len(tickers)} tickers S&P 500.")
    except Exception as e:
        print(f"Erreur lors de la récupération des tickers S&P 500: {e}")
    return tickers

In [5]:
# --- 2. Fonction de Scrapping (Stoxx 600 uniquement) ---

exchange_to_suffix_map = {
    'EURONEXT AMSTERDAM': '.AS',
    'ATHENS EXCHANGE': '.AT',
    'EURONEXT BRUSSELS': '.BR',
    'COPENHAGEN': '.CO',
    'DUB LIN': '.IR', # Note: DividendMax might list as 'Dublin' or similar
    'FRANKFURT STOCK EXCHANGE': '.DE', # Use the full scraped name
    'HELSINKI STOCK EXCHANGE': '.HE',    # Use the full scraped name
    'LISBON STOCK EXCHANGE': '.PT',      # Use the full scraped name
    'LONDON STOCK EXCHANGE': '.L',       # Use the full scraped name
    'MADRID STOCK EXCHANGE': '.MC',      # Use the full scraped name
    'MILAN STOCK EXCHANGE': '.MI',       # Use the full scraped name
    'OSLO STOCK EXCHANGE': '.OL',        # Use the full scraped name
    'EURONEXT PARIS': '.PA',             # Use the full scraped name
    'STOCKHOLM STOCK EXCHANGE': '.ST',   # Use the full scraped name
    'SIX SWISS EXCHANGE': '.SW',         # Use the full scraped name
    'VIENNA STOCK EXCHANGE': '.VI',      # Use the full scraped name
    'XETRA': '.DE',                     # Use the full scraped name (often appears as XETRA)
    'ITALIAN STOCK EXCHANGE': '.MI',     # Use the full scraped name (maps to Milan)
    'IRISH STOCK EXCHANGE': '.IR',       # Added based on AIBG
    'WARSAW STOCK EXCHANGE': '.WA',      # Added based on PEO, KGH, LPP, PGN, PKO, PZU
    'VALENCIA STOCK EXCHANGE': '.MC',    # Added based on SAB (assuming it uses Madrid suffix)
    'NEW YORK STOCK EXCHANGE': '',       # Added based on CNH, STLA (assuming no suffix needed for Yahoo)
    'BERLIN STOCK EXCHANGE': '.BE',      # Added based on KGX
    'LUXEMBOURG STOCK EXCHANGE': '.LU',  # Added based on RTLL
    'JOHANNESBURG STOCK EXCHANGE': '.JO' # Added based on OMU
}

def get_eurostoxx600_tickers():
    """
    Récupère la liste des tickers, noms et bourses de l'Eurostoxx 600
    depuis DividendMax.com, gérant la pagination.
    Détermine le ticker Yahoo Finance correct en utilisant un mappage des bourses
    et gérant certains cas particuliers de format de ticker (espaces, points).
    Retourne oune liste de dictionnaires [{'Ticker': '...', 'Nom': '...', 'YahooTicker': '...'}, ...].
    """
    print("Récupération des tickers Eurostoxx 600 depuis DividendMax (avec pagination)...")
    all_companies_data = []
    base_url = "https://www.dividendmax.com/market-index-constituents/stoxx600"
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; ColabEuroStoxx600Scraper/1.0)"
    }
    page_num = 1
    max_pages_to_check = 20 # Set a limit to avoid infinite loops

    # Access the global exchange_to_suffix_map
    global exchange_to_suffix_map

    while page_num <= max_pages_to_check:
        url = f"{base_url}?page={page_num}" # Assuming pagination uses '?page=' parameter
        print(f"  > Attempting to scrape page {page_num} from {url}")

        try:
            response = requests.get(url, headers=headers, timeout=20)
            response.raise_for_status()

            tables = pd.read_html(response.text)

            if not tables:
                print(f"  [Info] No tables found on page {page_num}. Assuming last page reached.")
                break # No tables found, likely the last page

            df = tables[0]
            if df.empty:
                 print(f"  [Info] Table on page {page_num} is empty. Assuming last page reached.")
                 break # Table is empty, likely the last page

            df.columns = [c.strip().lower() for c in df.columns]

            ticker_col_name = None
            name_col_name = None
            exchange_col_name = None

            # Identify column names - based on previous runs, 'ticker', 'company', and 'exchange' seem likely
            potential_ticker_cols = ["ticker", "symbol", "code", "ric"]
            potential_name_cols = ["company", "name"]
            potential_exchange_cols = ["exchange", "market"]


            for col in df.columns:
                if ticker_col_name is None and any(potential_name in col for potential_name in potential_ticker_cols):
                    ticker_col_name = col
                if name_col_name is None and any(potential_name in col for potential_name in potential_name_cols):
                    name_col_name = col
                if exchange_col_name is None and any(potential_name in col for potential_name in potential_exchange_cols):
                    exchange_col_name = col


            if ticker_col_name is None or name_col_name is None or exchange_col_name is None:
                print(f"  [Erreur] Colonnes 'ticker', 'company', ou 'exchange' non trouvées dans la table on page {page_num}.")
                print(f"  [DEBUG] Available columns on page {page_num}: {df.columns.tolist()}")
                break # Cannot find required columns, stop

            # Extract data from the current page and store as dictionaries
            page_companies_data = []
            for index, row in df.iterrows():
                ticker = str(row[ticker_col_name]).strip().upper()
                name = str(row[name_col_name]).strip()
                exchange = str(row[exchange_col_name]).strip().upper()

                if ticker and name and exchange: # Only add if all fields are not empty
                    # Clean ticker: remove trailing dot if present
                    if ticker.endswith('.'):
                        cleaned_ticker = ticker[:-1]
                        print(f"    - Cleaned trailing dot from ticker: {ticker} -> {cleaned_ticker}")
                    else:
                        cleaned_ticker = ticker

                    # 1. Use the mapping to find the Yahoo Finance suffix
                    yahoo_suffix = exchange_to_suffix_map.get(exchange, '')
                    if not yahoo_suffix and exchange != 'NEW YORK STOCK EXCHANGE': # Assuming NYSE has no suffix in this case
                         print(f"    - No Yahoo Finance suffix found for exchange: {exchange} (for ticker {ticker}). Using empty suffix.")


                    # 2. Construct the YahooTicker, handle specific cases like spaces
                    if ' ' in cleaned_ticker:
                        parts = cleaned_ticker.split(' ')
                        if len(parts) == 2:
                             yahoo_ticker = f"{parts[0]}-{parts[1]}{yahoo_suffix}"
                             print(f"    - Handling ticker with space: {cleaned_ticker} -> {yahoo_ticker}")
                        else: # Handle multiple spaces if necessary, or just use the cleaned ticker
                            yahoo_ticker = f"{cleaned_ticker}{yahoo_suffix}"
                            print(f"    - Handling ticker with multiple spaces or unusual format: {cleaned_ticker}. Using default suffix: {yahoo_ticker}")
                    else:
                        yahoo_ticker = f"{cleaned_ticker}{yahoo_suffix}"


                    # 3. Append a dictionary with Ticker, Nom, and YahooTicker
                    page_companies_data.append({
                        'Ticker': ticker, # Keep original ticker for reference
                        'Nom': name,
                        'YahooTicker': yahoo_ticker
                    })
                    # 5. Update print statement
                    print(f"    - Scraped {ticker} ({name}) from {exchange}. Generated YahooTicker: {yahoo_ticker}")


            # If no new companies were found on this page, it might be the last page
            if not page_companies_data:
                 print(f"  [Info] No new companies found on page {page_num}. Assuming last page reached.")
                 break

            all_companies_data.extend(page_companies_data)
            print(f"  > Successfully scraped {len(page_companies_data)} companies from page {page_num}. Total collected: {len(all_companies_data)}")

            page_num += 1 # Move to the next page

        except requests.exceptions.RequestException as e:
            print(f"  [Erreur] Erreur lors de la récupération de la page {page_num}: {e}")
            break # Stop if there's a request error
        except Exception as e:
            print(f"  [Erreur] Erreur lors du parsing de la page {page_num}: {e}")
            break # Stop if there's a parsing error

    # No need to remove duplicates here, as we expect unique companies per page,
    # and will handle potential duplicates later if needed.

    # 6. Update final print statement
    print(f"\nTerminé la récupération. Trouvé un total de {len(all_companies_data)} sociétés Eurostoxx 600 with Yahoo Tickers.")
    return all_companies_data # Return list of dictionaries

In [6]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

def fetch_historical_fundamentals(ticker_list, zone='USA'):
    historical_data_list = []
    total = len(ticker_list)

    for i, item in enumerate(ticker_list):
        ticker_str = None
        ticker_symbol = None
        nom = None

        if zone == 'USA':
            ticker_str = item
            ticker_symbol = item
            nom = item
            print(f"\nTraitement Historique ({i+1}/{total}): {ticker_str} (Zone: {zone})")

        elif zone == 'Eurostoxx':
            if isinstance(item, dict) and 'Ticker' in item and 'YahooTicker' in item:
                ticker_str = item.get('Ticker', '')
                ticker_symbol = item.get('YahooTicker', '')
                nom = item.get('Nom', ticker_str)
                print(f"\nTraitement Historique ({i+1}/{total}): {ticker_str} (Yahoo: {ticker_symbol}, Zone: {zone})")
            else:
                print(f"\n[Erreur] Format inattendu pour l'entrée {i+1}. Skip.")
                continue

        if not ticker_symbol:
             print(f"  [Erreur] Yahoo Ticker non disponible pour {ticker_str}. Skip.")
             continue

        try:
            ticker = yf.Ticker(ticker_symbol)

            # 1. Récupération métadonnées (Info)
            try:
                info_data = ticker.info
                sector_val = info_data.get('sector', 'N/A')
                industry_val = info_data.get('industry', 'N/A')
                country_val = info_data.get('country', 'N/A')
                beta_val = info_data.get('beta', None) 
                # Note: On ne prend plus marketCap ici car c'est la valeur actuelle
            except Exception as e_info:
                print(f"  [Warning] Info manquante pour {ticker_symbol}: {e_info}")
                sector_val = 'N/A'
                industry_val = 'N/A'
                country_val = 'N/A'
                beta_val = None

            # 2. Récupération États Financiers
            financials = ticker.financials
            balance_sheet = ticker.balance_sheet
            dividends = ticker.dividends 
            
            # --- NOUVEAU : Récupération de l'historique du nombre d'actions ---
            try:
                # On récupère l'historique complet des actions depuis 2020 pour couvrir la période
                shares_history = ticker.get_shares_full(start="2020-01-01")
                # Important : on retire la timezone pour pouvoir comparer avec les autres dates
                if shares_history is not None and not shares_history.empty:
                    if shares_history.index.tz is not None:
                        shares_history.index = shares_history.index.tz_localize(None)
            except Exception as e_shares:
                print(f"  [Warning] Impossible de récupérer l'historique des actions pour {ticker_symbol}: {e_shares}")
                shares_history = pd.Series(dtype='float64')

            if financials.empty or balance_sheet.empty:
                print(f"  [Info] Données financières manquantes pour {ticker_symbol}.")
                continue

            financials_t = financials.transpose()
            balance_sheet_t = balance_sheet.transpose()

            if financials_t.index.tz is not None:
                financials_t.index = financials_t.index.tz_localize(None)
            if balance_sheet_t.index.tz is not None:
                balance_sheet_t.index = balance_sheet_t.index.tz_localize(None)

            # 3. Boucle temporelle
            for period_date in financials_t.index:
                # Filtre 2021-2024
                if period_date.year < 2021 or period_date.year > 2024:
                    continue

                if period_date not in balance_sheet_t.index:
                    continue

                try:
                    # Extraction Raw Data
                    net_income = financials_t.loc[period_date].get('Net Income')
                    total_revenue = financials_t.loc[period_date].get('Total Revenue')
                    basic_eps = financials_t.loc[period_date].get('Basic EPS')

                    total_equity = balance_sheet_t.loc[period_date].get('Total Stockholder Equity')
                    if total_equity is None:
                         total_equity = balance_sheet_t.loc[period_date].get('Stockholders Equity')

                    total_assets = balance_sheet_t.loc[period_date].get('Total Assets')
                    total_debt = balance_sheet_t.loc[period_date].get('Total Debt')

                    # --- Prix de fin d'année ---
                    close_price = None
                    price_data_eod = ticker.history(start=period_date.normalize(), end=(period_date + pd.Timedelta(days=4)).normalize())
                    if not price_data_eod.empty and 'Close' in price_data_eod.columns:
                        close_price = price_data_eod['Close'].iloc[0]

                    # --- NOUVEAU : Calcul Market Cap Historique ---
                    historical_market_cap = None
                    shares_outstanding = None
                    
                    if close_price is not None and not shares_history.empty:
                        # .asof() trouve la valeur la plus récente à la date donnée (ex: nombre d'actions au 31 déc 2022)
                        try:
                            shares_outstanding = shares_history.asof(period_date)
                            if shares_outstanding and shares_outstanding > 0:
                                historical_market_cap = close_price * shares_outstanding
                        except Exception as e_mc:
                            pass # Si échec, reste None

                    # --- Dividendes ---
                    annual_dividends = None
                    if not dividends.empty:
                        year_start = pd.Timestamp(f'{period_date.year}-01-01').tz_localize(None)
                        year_end = pd.Timestamp(f'{period_date.year}-12-31').tz_localize(None)
                        dividends_filtered = dividends.copy()
                        if dividends_filtered.index.tz is not None:
                            dividends_filtered.index = dividends_filtered.index.tz_localize(None)
                        yearly_dividends = dividends_filtered.loc[(dividends_filtered.index >= year_start) & (dividends_filtered.index <= year_end)]
                        if not yearly_dividends.empty:
                            annual_dividends = yearly_dividends.sum()

                    # --- Volume ---
                    annual_volume_traded = None
                    try:
                        year_start_date = pd.Timestamp(f'{period_date.year}-01-01').tz_localize(None)
                        year_end_date = pd.Timestamp(f'{period_date.year}-12-31').tz_localize(None)
                        yearly_history = ticker.history(start=year_start_date, end=year_end_date)
                        if not yearly_history.empty and 'Volume' in yearly_history.columns:
                            annual_volume_traded = yearly_history['Volume'].sum()
                    except Exception:
                        pass

                    # --- Ratios (Mis à jour avec historical_market_cap) ---
                    roe = net_income / total_equity if total_equity and net_income and total_equity != 0 else None
                    roa = net_income / total_assets if total_assets and net_income and total_assets != 0 else None
                    
                    # NOTE: On utilise historical_market_cap ici au lieu de info['marketCap']
                    debt_to_equity = total_debt / historical_market_cap if total_debt and historical_market_cap and historical_market_cap != 0 else None
                    
                    per_calcule = close_price / basic_eps if close_price and basic_eps and basic_eps != 0 else None

                    data_entry = {
                        'Ticker': ticker_str,
                        'YahooTicker': ticker_symbol,
                        'Nom': nom,
                        'Zone': zone,
                        'Sector': sector_val,
                        'Industry': industry_val,
                        'Country': country_val,
                        'Beta': beta_val, 
                        'MarketCapitalization': historical_market_cap, # Valeur historique correcte
                        'SharesOutstanding': shares_outstanding, # Utile pour vérification
                        'AnneeFiscale': period_date.year,
                        'DateRapport': period_date.strftime('%Y-%m-%d'),
                        'NetIncome': net_income,
                        'TotalRevenue': total_revenue,
                        'TotalEquity': total_equity,
                        'TotalAssets': total_assets,
                        'TotalDebt': total_debt,
                        'EPS': basic_eps,
                        'EOD_Close_Price': close_price,
                        'ROE_calcule': roe,
                        'ROA_calcule': roa,
                        'DebtToEquity_calcule': debt_to_equity,
                        'PER_calcule': per_calcule,
                        'Dividendes_Annuels': annual_dividends,
                        'Annual_Volume_Traded': annual_volume_traded 
                    }
                    historical_data_list.append(data_entry)

                except Exception as e_inner:
                    print(f"  [Erreur] Extraction {period_date.year} pour {ticker_symbol}: {e_inner}")

            time.sleep(0.1)

        except Exception as e_outer:
            print(f"  [Grave Erreur] {ticker_symbol}: {e_outer}")

    if not historical_data_list:
        return None

    df_historical = pd.DataFrame(historical_data_list)

    columns_order = [
        'Ticker', 'YahooTicker', 'Nom', 'Zone',
        'AnneeFiscale', 'DateRapport',
        'Sector', 'Industry', 'Country', 'Beta', 'MarketCapitalization', 'SharesOutstanding',
        'PER_calcule', 'ROE_calcule', 'ROA_calcule', 'DebtToEquity_calcule',
        'EPS', 'EOD_Close_Price',
        'NetIncome', 'TotalRevenue', 'TotalEquity', 'TotalAssets', 'TotalDebt',
        'Dividendes_Annuels',
        'Annual_Volume_Traded' 
    ]

    final_columns = [col for col in columns_order if col in df_historical.columns]
    df_historical = df_historical[final_columns]
    df_historical = df_historical.sort_values(by=['Ticker', 'AnneeFiscale'], ascending=[True, False])
    return df_historical


In [7]:
# --- 4. Fonction de construction du csv Stoxx ---

def main_Stoxx():
    # Option pour Pandas : afficher toutes les lignes et colonnes lors du print
    pd.set_option('display.max_rows', None) # Utile pour le test
    pd.set_option('display.max_columns', None)

    print("--- Démarrage du script de collecte Eurostoxx600 (Historique) ---")

    # 1. Récupérer les tickers, noms, et Yahoo Tickers
    eurostoxx_companies_data = get_eurostoxx600_tickers()

    if not eurostoxx_companies_data:
        print("Aucun ticker Eurostoxx600 n'a été trouvé. Arrêt du script.")
        return

    # 2. Collecter les données historiques, passing the list of dictionaries and specifying the zone
    df_final = fetch_historical_fundamentals(eurostoxx_companies_data, zone='Eurostoxx') # Pass the list of dicts here

    if df_final is not None and not df_final.empty:

        # 3. Sauvegarder en CSV
        output_file = 'eurostoxx_historical_fundamentals.csv'
        df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\n--- Succès ---")
        print(f"Données historiques sauvegardées dans : {output_file}")

        # 4. Afficher le DataFrame complet dans la console (comme demandé)
        print("\n--- Aperçu du DataFrame Final (affiché avec display) ---")
        display(df_final)
        print("---------------------------------------------------------")

    else:
        print("\n--- Terminé ---")
        print("Aucune donnée historique n'a pu être collectée.")


In [8]:
# --- 5. Fonction de construction du csv S&P500 ---

def main_SP500():
    # Option pour Pandas : afficher toutes les lignes et colonnes lors du print
    pd.set_option('display.max_rows', None) # Utile pour le test
    pd.set_option('display.max_columns', None)

    print("--- Démarrage du script de collecte S&P 500 (Historique) ---")

    # 1. Récupérer les tickers
    sp500_tickers = get_sp500_tickers()

    if not sp500_tickers:
        print("Aucun ticker S&P 500 n'a été trouvé. Arrêt du script.")
        return

    # 2. Collecter les données historiques
    df_final = fetch_historical_fundamentals(sp500_tickers, zone='USA')

    if df_final is not None and not df_final.empty:

        # 3. Sauvegarder en CSV
        output_file = 'sp500_historical_fundamentals.csv'
        df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\n--- Succès ---")
        print(f"Données historiques sauvegardées dans : {output_file}")

        # 4. Afficher le DataFrame complet dans la console (comme demandé)
        print("\n--- Aperçu du DataFrame Final (affiché avec display) ---")
        display(df_final)
        print("---------------------------------------------------------")

    else:
        print("\n--- Terminé ---")
        print("Aucune donnée historique n'a pu être collectée.")

In [19]:
# --- 6. Construction des CSV ---
main_SP500()

--- Démarrage du script de collecte S&P 500 (Historique) ---
Récupération des tickers S&P 500...
Trouvé 502 tickers S&P 500.

Traitement Historique (1/502): MMM (Zone: USA)

Traitement Historique (2/502): AOS (Zone: USA)

Traitement Historique (3/502): ABT (Zone: USA)

Traitement Historique (4/502): ABBV (Zone: USA)

Traitement Historique (5/502): ACN (Zone: USA)

Traitement Historique (6/502): ADBE (Zone: USA)

Traitement Historique (7/502): AMD (Zone: USA)

Traitement Historique (8/502): AES (Zone: USA)

Traitement Historique (9/502): AFL (Zone: USA)

Traitement Historique (10/502): A (Zone: USA)

Traitement Historique (11/502): APD (Zone: USA)

Traitement Historique (12/502): ABNB (Zone: USA)

Traitement Historique (13/502): AKAM (Zone: USA)

Traitement Historique (14/502): ALB (Zone: USA)

Traitement Historique (15/502): ARE (Zone: USA)

Traitement Historique (16/502): ALGN (Zone: USA)

Traitement Historique (17/502): ALLE (Zone: USA)

Traitement Historique (18/502): LNT (Zone: USA

$CEG: possibly delisted; no price data found  (1d 2021-12-31 00:00:00 -> 2022-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1641272400")
$CEG: possibly delisted; no price data found  (1d 2021-01-01 00:00:00 -> 2021-12-31 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1609477200, endDate = 1640926800")



Traitement Historique (122/502): COO (Zone: USA)

Traitement Historique (123/502): CPRT (Zone: USA)

Traitement Historique (124/502): GLW (Zone: USA)

Traitement Historique (125/502): CPAY (Zone: USA)

Traitement Historique (126/502): CTVA (Zone: USA)

Traitement Historique (127/502): CSGP (Zone: USA)

Traitement Historique (128/502): COST (Zone: USA)

Traitement Historique (129/502): CTRA (Zone: USA)

Traitement Historique (130/502): CRWD (Zone: USA)

Traitement Historique (131/502): CCI (Zone: USA)

Traitement Historique (132/502): CSX (Zone: USA)

Traitement Historique (133/502): CMI (Zone: USA)

Traitement Historique (134/502): CVS (Zone: USA)

Traitement Historique (135/502): DHR (Zone: USA)

Traitement Historique (136/502): DRI (Zone: USA)

Traitement Historique (137/502): DDOG (Zone: USA)

Traitement Historique (138/502): DVA (Zone: USA)

Traitement Historique (139/502): DAY (Zone: USA)

Traitement Historique (140/502): DECK (Zone: USA)

Traitement Historique (141/502): DE (Zon

$GEHC: possibly delisted; no price data found  (1d 2021-12-31 00:00:00 -> 2022-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1641272400")
$GEHC: possibly delisted; no price data found  (1d 2021-01-01 00:00:00 -> 2021-12-31 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1609477200, endDate = 1640926800")



Traitement Historique (209/502): GEV (Zone: USA)


...


Traitement Historique (419/502): SO (Zone: USA)

Traitement Historique (420/502): LUV (Zone: USA)

Traitement Historique (421/502): SWK (Zone: USA)

Traitement Historique (422/502): SBUX (Zone: USA)

Traitement Historique (423/502): STT (Zone: USA)

Traitement Historique (424/502): STLD (Zone: USA)

Traitement Historique (425/502): STE (Zone: USA)

Traitement Historique (426/502): SYK (Zone: USA)

Traitement Historique (427/502): SMCI (Zone: USA)

Traitement Historique (428/502): SYF (Zone: USA)

Traitement Historique (429/502): SNPS (Zone: USA)

Traitement Historique (430/502): SYY (Zone: USA)

Traitement Historique (431/502): TMUS (Zone: USA)

Traitement Historique (432/502): TROW (Zone: USA)

Traitement Historique (433/502): TTWO (Zone: USA)

Traitement Historique (434/502): TPR (Zone: USA)

Traitement Historique (435/502): TRGP (Zone: USA)

Traitement Historique (436/502): TGT (Zone: USA)

Traitement Historique (437/502): TEL (Zone: USA)

Traitement Historique (438/502): TDY (Zone

$VLTO: possibly delisted; no price data found  (1d 2022-12-31 00:00:00 -> 2023-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1672808400")
$VLTO: possibly delisted; no price data found  (1d 2022-01-01 00:00:00 -> 2022-12-31 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1641013200, endDate = 1672462800")
$VLTO: possibly delisted; no price data found  (1d 2021-12-31 00:00:00 -> 2022-01-04 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1641272400")
$VLTO: possibly delisted; no price data found  (1d 2021-01-01 00:00:00 -> 2021-12-31 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1609477200, endDate = 1640926800")



Traitement Historique (469/502): VRSN (Zone: USA)

Traitement Historique (470/502): VRSK (Zone: USA)

Traitement Historique (471/502): VZ (Zone: USA)

Traitement Historique (472/502): VRTX (Zone: USA)

Traitement Historique (473/502): VTRS (Zone: USA)

Traitement Historique (474/502): VICI (Zone: USA)

Traitement Historique (475/502): V (Zone: USA)

Traitement Historique (476/502): VST (Zone: USA)

Traitement Historique (477/502): VMC (Zone: USA)

Traitement Historique (478/502): WRB (Zone: USA)

Traitement Historique (479/502): GWW (Zone: USA)

Traitement Historique (480/502): WAB (Zone: USA)

Traitement Historique (481/502): WMT (Zone: USA)

Traitement Historique (482/502): DIS (Zone: USA)

Traitement Historique (483/502): WBD (Zone: USA)

Traitement Historique (484/502): WM (Zone: USA)

Traitement Historique (485/502): WAT (Zone: USA)

Traitement Historique (486/502): WEC (Zone: USA)

Traitement Historique (487/502): WFC (Zone: USA)

Traitement Historique (488/502): WELL (Zone: USA

,Ticker,YahooTicker,Nom,Zone,AnneeFiscale,DateRapport,Sector,Industry,Country,Beta,MarketCapitalization,SharesOutstanding,PER_calcule,ROE_calcule,ROA_calcule,DebtToEquity_calcule,EPS,EOD_Close_Price,NetIncome,TotalRevenue,TotalEquity,TotalAssets,TotalDebt,Dividendes_Annuels,Annual_Volume_Traded
36,A,A,A,USA,2024,2024-10-31,Healthcare,Diagnostics & Research,United States,1.274,3.714045e+10,2.873280e+08,29.112952,0.218549,0.108813,0.091275,4.440000,129.261505,1.289000e+09,6.510000e+09,5.898000e+09,1.184600e+10,3.390000e+09,0.956000,4.310391e+08
37,A,A,A,USA,2023,2023-10-31,Healthcare,Diagnostics & Research,United States,1.274,2.979930e+10,2.925870e+08,24.134516,0.212147,0.115210,0.091781,4.220000,101.847656,1.240000e+09,6.833000e+09,5.845000e+09,1.076300e+10,2.735000e+09,0.911000,4.612575e+08
38,A,A,A,USA,2022,2022-10-31,Healthcare,Diagnostics & Research,United States,1.274,4.007099e+10,2.960410e+08,32.304590,0.236381,0.119066,0.069102,4.190000,135.356232,1.254000e+09,6.848000e+09,5.305000e+09,1.053200e+10,2.769000e+09,1.065000,4.272828e+08
39,A,A,A,USA,2021,2021-10-31,Healthcare,Diagnostics & Research,United States,1.274,4.622082e+10,3.027230e+08,38.362697,0.224531,0.113031,0.059043,3.980000,152.683533,1.210000e+09,6.319000e+09,5.389000e+09,1.070500e+10,2.729000e+09,0.776000,4.077569e+08
152,AAPL,AAPL,AAPL,USA,2024,2024-09-30,Technology,Consumer Electronics,United States,1.107,3.533550e+12,1.525080e+10,37.920790,1.645935,0.256825,0.030176,6.110000,231.696030,9.373600e+10,3.910350e+11,5.695000e+10,3.649800e+11,1.066290e+11,0.990000,1.435143e+10
153,AAPL,AAPL,AAPL,USA,2023,2023-09-30,Technology,Consumer Electronics,United States,1.107,2.687488e+12,1.563420e+10,27.905521,1.560760,0.275098,0.041335,6.160000,171.898010,9.699500e+10,3.832850e+11,6.214600e+10,3.525830e+11,1.110880e+11,0.950000,1.480589e+10
154,AAPL,AAPL,AAPL,USA,2022,2022-09-30,Technology,Consumer Electronics,United States,1.107,2.184353e+12,1.607080e+10,22.100909,1.969589,0.282924,0.060650,6.150000,135.920593,9.980300e+10,3.943280e+11,5.067200e+10,3.527550e+11,1.324800e+11,0.910000,2.206550e+10
155,AAPL,AAPL,AAPL,USA,2021,2021-09-30,Technology,Consumer Electronics,United States,1.107,2.287625e+12,1.653020e+10,NaN,NaN,NaN,NaN,NaN,138.390656,NaN,NaN,NaN,NaN,NaN,0.865000,2.274814e+10
12,ABBV,ABBV,ABBV,USA,2024,2024-12-31,Healthcare,Drug Manufacturers - General,United States,0.349,3.034118e+11,1.767140e+09,70.848170,1.286617,0.031651,0.221297,2.423443,171.696503,4.278000e+09,5.633400e+10,3.325000e+09,1.351610e+11,6.714400e+10,6.200000,1.398188e+09
13,ABBV,ABBV,ABBV,USA,2023,2023-12-31,Healthcare,Drug Manufacturers - General,United States,0.349,2.630210e+11,1.765540e+09,54.098429,0.469402,0.036100,0.225780,2.753773,148.974792,4.863000e+09,5.431800e+10,1.036000e+10,1.347110e+11,5.938500e+10,5.920000,1.375884e+09


---------------------------------------------------------


In [ ]:
# Construction Stoxx
main_Stoxx()

--- Démarrage du script de collecte Eurostoxx600 (Historique) ---
Récupération des tickers Eurostoxx 600 depuis DividendMax (avec pagination)...
  > Attempting to scrape page 1 from https://www.dividendmax.com/market-index-constituents/stoxx600?page=1


/tmp/ipykernel_53538/597466302.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
/tmp/ipykernel_53538/597466302.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


    - Scraped 1U1 (1&1 DrillischAktiengesellschaft) from FRANKFURT STOCK EXCHANGE. Generated YahooTicker: 1U1.DE
    - Scraped III (3i Group plc) from LONDON STOCK EXCHANGE. Generated YahooTicker: III.L
    - Scraped A2A (A2A Spa) from ITALIAN STOCK EXCHANGE. Generated YahooTicker: A2A.MI
    - Scraped AAK (AAK AB) from STOCKHOLM STOCK EXCHANGE. Generated YahooTicker: AAK.ST
    - Scraped AALB (Aalberts NV) from EURONEXT AMSTERDAM. Generated YahooTicker: AALB.AS
    - Scraped ARL (Aareal Beteiligungen AG) from FRANKFURT STOCK EXCHANGE. Generated YahooTicker: ARL.DE
    - Scraped ABBN (ABB Ltd.) from SIX SWISS EXCHANGE. Generated YahooTicker: ABBN.SW
    - Scraped ABDN (Aberdeen Group Plc) from LONDON STOCK EXCHANGE. Generated YahooTicker: ABDN.L
    - Scraped ABN (ABN AMRO Bank N.V. - NLDR) from EURONEXT AMSTERDAM. Generated YahooTicker: ABN.AS
    - Scraped AC (Accor) from EURONEXT PARIS. Generated YahooTicker: AC.PA
    - Scraped ACKB (Ackermans & van Haaren NV) from EURONEXT BRUSSEL

/tmp/ipykernel_53538/597466302.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
/tmp/ipykernel_53538/597466302.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


    - Cleaned trailing dot from ticker: BA. -> BA
    - Scraped BA. (BAE Systems plc) from LONDON STOCK EXCHANGE. Generated YahooTicker: BA.L
    - Scraped BBY (Balfour Beatty plc) from LONDON STOCK EXCHANGE. Generated YahooTicker: BBY.L
    - Scraped BALN (Baloise Holding AG) from SIX SWISS EXCHANGE. Generated YahooTicker: BALN.SW
    - Scraped BBVA (Banco Bilbao Vizcaya Argentaria S.A.) from MADRID STOCK EXCHANGE. Generated YahooTicker: BBVA.MC
    - Scraped BAMI (Banco BPM S.p.A) from ITALIAN STOCK EXCHANGE. Generated YahooTicker: BAMI.MI
    - No Yahoo Finance suffix found for exchange: EURONEXT LISBON (for ticker BCP). Using empty suffix.
    - Scraped BCP (Banco Comercial Portugues, S.A.) from EURONEXT LISBON. Generated YahooTicker: BCP
    - Scraped SAB (Banco de Sabadell, S.A.) from VALENCIA STOCK EXCHANGE. Generated YahooTicker: SAB.MC
    - Scraped SAN (Banco Santander S.A.) from MADRID STOCK EXCHANGE. Generated YahooTicker: SAN.MC
    - Scraped BKT (Bankinter SA) from MADRID

...

  [Info] Données financières manquantes pour RTLL.LU.

Traitement Historique (420/565): RUI (Yahoo: RUI.PA, Zone: Eurostoxx)

Traitement Historique (421/565): RWE (Yahoo: RWE.DE, Zone: Eurostoxx)

Traitement Historique (422/565): RYA (Yahoo: RYA.IR, Zone: Eurostoxx)

Traitement Historique (423/565): SAAB B (Yahoo: SAAB-B.ST, Zone: Eurostoxx)

Traitement Historique (424/565): SAF (Yahoo: SAF.PA, Zone: Eurostoxx)

Traitement Historique (425/565): SGE (Yahoo: SGE.L, Zone: Eurostoxx)

Traitement Historique (426/565): SBRY (Yahoo: SBRY.L, Zone: Eurostoxx)

Traitement Historique (427/565): SPM (Yahoo: SPM.MI, Zone: Eurostoxx)

Traitement Historique (428/565): SALM (Yahoo: SALM.OL, Zone: Eurostoxx)

Traitement Historique (429/565): SAMPO (Yahoo: SAMPO.HE, Zone: Eurostoxx)

Traitement Historique (430/565): SAND (Yahoo: SAND.ST, Zone: Eurostoxx)

Traitement Historique (431/565): SAN (Yahoo: SAN.PA, Zone: Eurostoxx)

Traitement Historique (432/565): SPL (Yahoo: SPL.WA, Zone: Eurostoxx)

Traiteme

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SESGL.LU"}}}
$SESGL.LU: possibly delisted; no timezone found


  [Info] Données financières manquantes pour SESGL.LU.

Traitement Historique (446/565): SVT (Yahoo: SVT.L, Zone: Eurostoxx)

Traitement Historique (447/565): SGSN (Yahoo: SGSN.SW, Zone: Eurostoxx)

Traitement Historique (448/565): SIE (Yahoo: SIE.DE, Zone: Eurostoxx)

Traitement Historique (449/565): ENR (Yahoo: ENR.DE, Zone: Eurostoxx)

Traitement Historique (450/565): SGRE (Yahoo: SGRE.MC, Zone: Eurostoxx)


$SGRE.MC: possibly delisted; no timezone found


  [Info] Données financières manquantes pour SGRE.MC.

Traitement Historique (451/565): SHL (Yahoo: SHL.DE, Zone: Eurostoxx)

Traitement Historique (452/565): LIGHT (Yahoo: LIGHT.AS, Zone: Eurostoxx)

Traitement Historique (453/565): SIKA (Yahoo: SIKA.SW, Zone: Eurostoxx)

Traitement Historique (454/565): WAF (Yahoo: WAF.DE, Zone: Eurostoxx)

Traitement Historique (455/565): SIM (Yahoo: SIM, Zone: Eurostoxx)

Traitement Historique (456/565): SEB A (Yahoo: SEB-A.ST, Zone: Eurostoxx)

Traitement Historique (457/565): SKA B (Yahoo: SKA-B.ST, Zone: Eurostoxx)

Traitement Historique (458/565): SKF B (Yahoo: SKF-B.ST, Zone: Eurostoxx)

Traitement Historique (459/565): SN. (Yahoo: SN.L, Zone: Eurostoxx)

Traitement Historique (460/565): SMIN (Yahoo: SMIN.L, Zone: Eurostoxx)

Traitement Historique (461/565): SRG (Yahoo: SRG.MI, Zone: Eurostoxx)

Traitement Historique (462/565): GLE (Yahoo: GLE.PA, Zone: Eurostoxx)

Traitement Historique (463/565): SW (Yahoo: SW.PA, Zone: Eurostoxx)

Traitement